# `Setup`

In [1]:
import pandas as pd
import numpy as np
import os

try:
    # Mounting Colab Drive if possible
    from google.colab import drive
    drive.mount('/content/drive')

    # Cloning repo for colab
    !git clone https://github.com/RasKrebs/aml_itu
    %cd aml_itu/
    os.environ["COLAB"] = "True"

except:
    # Changing directory into aml_itu
    if os.getcwd().split('/')[-1] != 'aml_itu': os.chdir(os.path.abspath('.').split('aml_itu/')[0]+'aml_itu')
    !git pull origin main
    os.environ["COLAB"] = "False"

# Utils Import
from utils.helpers import *
from utils.StatefarmPytorchDataset import StateFarmDataset

# Printing current working directory
print(os.getcwd())

Mounted at /content/drive
Cloning into 'aml_itu'...
remote: Enumerating objects: 169, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 169 (delta 3), reused 1 (delta 0), pack-reused 144
Receiving objects: 100% (169/169), 178.22 MiB | 29.75 MiB/s, done.
Resolving deltas: 100% (68/68), done.
/content/aml_itu
/content/aml_itu


### `Config`

In [2]:
# Loading the config file (if content is in workin directory must mean colab is being used)
config = load_config(eval(os.environ["COLAB"]))


# Training Images
train_img = config['dataset']['images']['train']

# Outputting config
config

{'dataset': {'name': 'state-farm-distracted-driver-detection',
  'colab_path': '/content/drive/MyDrive/aml-distracted-drivers-project',
  'data': '/content/drive/MyDrive/aml-distracted-drivers-project/state-farm-distracted-driver-detection/driver_imgs_list.csv',
  'images': {'train': '/content/drive/MyDrive/aml-distracted-drivers-project/state-farm-distracted-driver-detection/imgs/train',
   'test': '/content/drive/MyDrive/aml-distracted-drivers-project/state-farm-distracted-driver-detection/imgs/test'},
  'class_mapping': {'c0': 'safe driving',
   'c1': 'texting - right',
   'c2': 'talking on the phone - right',
   'c3': 'texting - left',
   'c4': 'talking on the phone - left',
   'c5': 'operating the radio',
   'c6': 'drinking',
   'c7': 'reaching behind',
   'c8': 'hair and makeup',
   'c9': 'talking to passenger'}},
 'outputs': {'path': './outputs'},
 'modeling_params': {'batch_size': 32, 'epochs': 100}}

## `Main`

In [5]:
from torchvision.transforms import Compose, Resize, Normalize, Lambda
import torchvision.transforms.functional as TF
from torch import float32

def custom_transforms(image):
    # Convert image to float
    image = image.to(float32)
    # Resize image
    image = TF.resize(image, (224, 224))
    # Normalize
    image = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])(image)

    return image

# Combine custom transformations
transform = Compose([Lambda(custom_transforms)])

# Creating the dataset with custom transformations
train_data = StateFarmDataset(config, split='train', transform=transform)
test_data = StateFarmDataset(config, split='test', transform=transform)

In [21]:
# Data Augmentation
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
])


# Creating the dataset with custom transformations
train_data = StateFarmDataset(config, split='train', transform=train_transforms)

In [22]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from utils.StatefarmPytorchDataset import StateFarmDataset

In [ ]:
!pip install timm

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import timm
from torch.utils.data import DataLoader
from tqdm import tqdm

label_map = {'c0': 0, 'c1': 1, 'c2': 2, 'c3': 3, 'c4': 4, 'c5': 5, 'c6': 6, 'c7': 7, 'c8': 8, 'c9': 9}


# Load the EfficientNet model
model_name = 'efficientnet_b5'  # You can choose from b0 to b7
efficientnet_model = timm.create_model(model_name, pretrained=True)

# Freeze the early layers
for param in efficientnet_model.parameters():
    param.requires_grad = False

# Replace the classifier layer
num_ftrs = efficientnet_model.classifier.in_features
efficientnet_model.classifier = nn.Linear(num_ftrs, 10)  # Assuming 10 classes

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
efficientnet_model = efficientnet_model.to(device)

# Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(efficientnet_model.parameters(), lr=0.001, momentum=0.9)

# DataLoader, batch size and worker threads

train_loader = DataLoader(train_data, batch_size=config['modeling_params']['batch_size'], shuffle=True, num_workers = 4)
test_loader = DataLoader(test_data, batch_size=config['modeling_params']['batch_size'], shuffle=False)


# Training function
def train_model(model, criterion, optimizer, train_loader, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        for images, label_tuple in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            labels = [label_map[label] for label in label_tuple]
            labels = torch.tensor(labels, dtype=torch.long)  # Convert list of integers to a tensor
            images, labels = images.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            loss.backward()
            optimizer.step()

            # Calculate accuracy
            _, predicted = torch.max(outputs.data, 1)
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = 100 * correct_predictions / total_predictions
        print(f'Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')


# Example of calling the training function
train_model(efficientnet_model, criterion, optimizer, train_loader, num_epochs=10)


model.safetensors:   0%|          | 0.00/122M [00:00<?, ?B/s]

Epoch 1/10: 100%|██████████| 565/565 [02:07<00:00,  4.42it/s]


Loss: 1.3391, Accuracy: 70.27%


Epoch 2/10: 100%|██████████| 565/565 [02:08<00:00,  4.40it/s]


Loss: 0.6809, Accuracy: 89.11%


...

Loss: 0.2520, Accuracy: 95.58%


Epoch 9/10: 100%|██████████| 565/565 [02:08<00:00,  4.39it/s]


Loss: 0.2323, Accuracy: 96.24%


Epoch 10/10: 100%|██████████| 565/565 [02:08<00:00,  4.38it/s]

Loss: 0.2156, Accuracy: 96.45%


In [16]:
import torch
from torch.utils.data import DataLoader

def test_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():  # No need to track gradients during testing
        for images, label_tuple in test_loader:
            labels = [label_map[label] for label in label_tuple]
            labels = torch.tensor(labels, dtype=torch.long)
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

    accuracy = 100 * correct_predictions / total_predictions
    print(f'Accuracy on test set: {accuracy:.2f}%')

# Test the model
test_model(efficientnet_model, test_loader)


Accuracy on test set: 54.29%


NEW MODEL


In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
import timm
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm import tqdm

# Model Setup
model_name = 'efficientnet_b1'
model = timm.create_model(model_name, pretrained=True)
num_ftrs = model.classifier.in_features
model.classifier = nn.Sequential(
    nn.Dropout(0.7),
    nn.Linear(num_ftrs, 10)
)

# Loss Function, Optimizer, and Scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=2e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Device Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Early Stopping Helper
class EarlyStopping:
    def __init__(self, patience=5, verbose=False):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss, model):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

early_stopping = EarlyStopping(patience=5, verbose=True)

# Training Loop
def train_one_epoch(model, train_loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    for images, label_tuple in train_loader:
        labels = [label_map[label] for label in label_tuple]
        labels = torch.tensor(labels, dtype=torch.long)  # Convert list of integers to a tensor
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(train_loader)

def validate_model(model, test_loader, criterion):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for images, label_tuple in test_loader:
            labels = [label_map[label] for label in label_tuple]
            labels = torch.tensor(labels, dtype=torch.long)  # Convert list of integers to a tensor
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
    return running_loss / len(test_loader)

num_epochs = 25
for epoch in range(num_epochs):
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion)
    val_loss = validate_model(model, test_loader, criterion)
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

    early_stopping(val_loss, model)
    if early_stopping.early_stop:
        print("Early stopping")
        break

    scheduler.step()


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - P

Epoch 1/25, Train Loss: 0.2064, Val Loss: 0.6483
Epoch 2/25, Train Loss: 0.0315, Val Loss: 0.4985
Epoch 3/25, Train Loss: 0.0213, Val Loss: 0.5833
Epoch 4/25, Train Loss: 0.0212, Val Loss: 0.4472
Epoch 5/25, Train Loss: 0.0225, Val Loss: 0.6800
Epoch 6/25, Train Loss: 0.0079, Val Loss: 0.5513
Epoch 7/25, Train Loss: 0.0129, Val Loss: 0.5934
Epoch 8/25, Train Loss: 0.0057, Val Loss: 0.4157
Epoch 9/25, Train Loss: 0.0012, Val Loss: 0.4703
Epoch 10/25, Train Loss: 0.0008, Val Loss: 0.4331
Epoch 11/25, Train Loss: 0.0006, Val Loss: 0.4966
Epoch 12/25, Train Loss: 0.0004, Val Loss: 0.4828
Epoch 13/25, Train Loss: 0.0003, Val Loss: 0.4632
Early stopping


In [24]:

def test_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():  # No need to track gradients during testing
        for images, label_tuple in test_loader:
            labels = [label_map[label] for label in label_tuple]
            labels = torch.tensor(labels, dtype=torch.long)
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

    accuracy = 100 * correct_predictions / total_predictions
    print(f'Accuracy on test set: {accuracy:.2f}%')

# Test the model
test_model(model, test_loader)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Accuracy on test set: 89.69%


In [25]:
torch.save(model.state_dict(), 'model_weights_89acc.pth')
